In [4]:
import pandas as pd

# 读取 CSV 文件
file_path = '../data/csv/YanAn_delay_data.csv'
df = pd.read_csv(file_path)

# 解析时间戳并提取日期
df['date'] = pd.to_datetime(df['time'], unit='s').dt.date

# 排除异常值
# 过滤掉 differential_delay 列中小于等于0的值
df_filtered = df[df['differential_delay'] > 0]
print(df_filtered['differential_delay'].describe())

# 使用分位数方法定义合理范围，去除异常值
q_low = df_filtered['differential_delay'].quantile(0.2)
q_high = df_filtered['differential_delay'].quantile(0.82)

# 过滤掉超出合理范围的数据
df_filtered_no_outliers = df_filtered[(df_filtered['differential_delay'] >= q_low) & (df_filtered['differential_delay'] <= q_high)]

# 按日期计算平均值
daily_avg_df = df_filtered_no_outliers.groupby(['station_id', 'station_name', 'date']).agg({'differential_delay': 'mean'}).reset_index()

# 列名station_name改为district_name
daily_avg_df.rename(columns={'station_name': 'district_name'}, inplace=True)

# 保存结果到新的 CSV 文件
output_file = '../data/csv/delay_avg_data.csv'
daily_avg_df.to_csv(output_file, index=False)

count    9.335983e+06
mean     1.507996e+08
std      3.504375e+08
min      1.480000e-01
25%      1.101857e+03
50%      1.150636e+03
75%      1.318195e+03
max      9.998571e+08
Name: differential_delay, dtype: float64
